In [1]:
import sys

from FileTools import FileTools
# from pprint import pprint
from pathlib import Path
from zipfile import ZipFile
import pickle
#import random
import json
import time
import os

#from matplotlib import pyplot as plt
#import seaborn as sns
import pandas as pd
import numpy as np
import math
import re

from kaggle.api.kaggle_api_extended import KaggleApi

In [2]:
# authenticate with API

currWorkDir = os.getcwd()
userDir = Path.home()
keyPath = f"{userDir}\\PYC\\ADMIN\\kaggle.json"

with open( keyPath, 'r' ) as f: keyDict = json.load( f )
userTitle, keyTitle = keyDict.keys()
kaggleUsername, kaggleKey = keyDict[ userTitle ], keyDict[ keyTitle ]

os.environ[ 'KAGGLE_USERNAME' ] = kaggleUsername
os.environ[ 'KAGGLE_KEY' ] = kaggleKey

api = KaggleApi()
api.authenticate()


In [3]:
# retrieve dataset
datasetOwner = 'lucafrance'
datasetName = 'the-world-factbook-by-cia'
api.dataset_download_files( f'{datasetOwner}/{datasetName}', path="." )

# await download
fTools = FileTools()
datasetFName = None
print( "Waiting for dataset download" )
while True:
    time.sleep( 1 )
    sortedFs = fTools.datesortFiles( currWorkDir, datasetName )
    if len( sortedFs ) == 0: continue
    datasetFName = list( sortedFs )[ 0 ]
    print( f"Latest: {datasetFName}" )
    break

Waiting for dataset download
Latest: the-world-factbook-by-cia.zip


In [4]:
# extract and identify datafiles

origDataDir = f"{currWorkDir}\\data_or"
if not os.path.exists( origDataDir ): os.makedirs( origDataDir )

if datasetFName and Path( datasetFName ).suffix == ".zip":
    with ZipFile( datasetFName, 'r' ) as zipF: zipF.extractall( origDataDir )

dataPaths = [ f"{origDataDir}\\{pth}" for pth in os.listdir( origDataDir )
    if Path( pth ).suffix == ".csv" ]

if len( dataPaths )>0:
    dffBook = pd.read_csv( [ pth for pth in dataPaths ][ 0 ] )
    print( "Got dffBook DF from extracted dataset at:\n", dataPaths[ 0 ] )
else: print( "Failed get CSV" ); sys.exit()

dffBook

Got dffBook DF from extracted dataset at:
 C:\Users\romstroller\PYC\ciaFactBook\data_or\countries.csv


,Country,url,Introduction: Background,Geography: Location,Geography: Geographic coordinates,Geography: Map references,Geography: Area - total,Geography: Area - land,Geography: Area - water,Geography: Area - comparative,...,Transportation: Waterways - note 2,Transportation: Waterways - top ten largest natural lakes (by surface area),Transportation: Waterways - note 3,Transportation: Ports and terminals - top twenty container ports as measured by Twenty-Foot Equivalent Units (TEUs) throughput,Transnational Issues: Refugees and internally displaced persons,Transnational Issues: Trafficking in persons - Tier 2 Watch List,Transnational Issues: Trafficking in persons - Tier 3,Transnational Issues: Illicit drugs - cocaine,Transnational Issues: Illicit drugs - opiates,Energy: Electricity access - population without electricity
0,Afghanistan,https://www.cia.gov/the-world-factbook/countri...,Ahmad Shah DURRANI unified the Pashtun tribes ...,"Southern Asia, north and west of Pakistan, eas...","33 00 N, 65 00 E",Asia,"652,230 sq km","652,230 sq km",0 sq km,almost six times the size of Virginia; slightl...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Akrotiri,https://www.cia.gov/the-world-factbook/countri...,By terms of the 1960 Treaty of Establishment t...,"Eastern Mediterranean, peninsula on the southw...","34 37 N, 32 58 E",Middle East,123 sq km,NaN,NaN,"about 0.7 times the size of Washington, DC",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Albania,https://www.cia.gov/the-world-factbook/countri...,Albania declared its independence from the Ott...,"Southeastern Europe, bordering the Adriatic Se...","41 00 N, 20 00 E",Europe,"28,748 sq km","27,398 sq km","1,350 sq km",slightly smaller than Maryland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Algeria,https://www.cia.gov/the-world-factbook/countri...,Algeria has known many empires and dynasties s...,"Northern Africa, bordering the Mediterranean S...","28 00 N, 3 00 E",Africa,"2,381,740 sq km","2,381,740 sq km",0 sq km,slightly less than 3.5 times the size of Texas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,American Samoa,https://www.cia.gov/the-world-factbook/countri...,Tutuila was settled by 1000 B.C. and the islan...,"Oceania, group of islands in the South Pacific...","14 20 S, 170 00 W",Oceania,224 sq km,224 sq km,0 sq km,"slightly larger than Washington, DC",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,West Bank,https://www.cia.gov/the-world-factbook/countri...,The landlocked West Bank - the larger of the t...,"Middle East, west of Jordan, east of Israel","32 00 N, 35 15 E",Middle East,"5,860 sq km","5,640 sq km",220 sq km,slightly smaller than Delaware,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254,World,https://www.cia.gov/the-world-factbook/countri...,"Globally, the 20th century was marked by: (a) ...",NaN,NaN,Physical Map of the World,510.072 million sq km,148.94 million sq km,361.9 million sq km,land area about 16 times the size of the US,...,there are 20 countries without rivers: 3 in Af...,"Caspian Sea (Azerbaijan, Iran, Kazakhstan, Rus...","the deepest lake in the world (1,620 m), and a...","Shanghai (China) - 43,303,000; Singapore (Sing...",the UN High Commissioner for Refugees (UNHCR) ...,"(44 countries) Armenia, Aruba, Azerbaijan, Bar...","(19 countries) Afghanistan, Algeria, Belarus, ...",worldwide coca cultivation in 2020 likely amou...,worldwide illicit opium poppy cultivation prob...,NaN
255,Yemen,https://www.cia.gov/the-world-factbook/countri...,The Kingdom of Yemen (colloquially known as No...,"Middle East, bordering the Arabian Sea, Gulf o...","15 00 N, 48 00 E",Middle East,"527,968 sq km","527,968 sq km",0 sq km,almost four times the size of Alabama; slightl...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16 million (2019)
256,Zambia,https://www.cia.gov/the-world-factbook/countri...,Multiple waves of Bantu-speaking groups moved ...,"Southern Africa, east of Angola, south of 

In [5]:
# Analyse cell data for numbers and units

def matchNumbers( df_in, coIdex, patrn ):
    # get any number-pattern match from each row in a list
    return df_in.iloc[ :, coIdex ].str.findall( patrn )


def getRemainder( df_in, coIdex, pattrn ):
    # store non-number remainder of string (potential unit etc)
    return df_in.iloc[ :, coIdex ].str.replace( pattrn, '' )


# MATCHING NUMBERS
# REGEX:
#   capture group             (                 
#   zero/one                  [+-]?             possible number sign
#   1-3 nums                  \d{1,3}           up to three straight nums
#   non-capture subgroup      (?:               possible thousand-groups
#     comma and 3 nums          ,\d{3}          (sep. comma)
#     zero or more times        )*              
#   non-capture subgroup      (?:               then possible decimal part
#     decimal and 1+nums        \.\d+           
#     zero/one time             )?            
#   OR (alt. to last seq)     |                 or no groups, just              
#     0+ nums, dec, 1+nums      \d*\.\d+        more nums and poss decimal
#   OR (alt. to last seq)     |             
#     1+ nums                   \d+             or just more numbers.
#   Close capture group       )
#   ( match basic number last to capture greatest valid str segment )

patt = re.compile( r'([+-]?\d{1,3}(?:,\d{3})*(?:\.\d+)?|\d*\.\d+|\d+)' )

dfFbDict = { }  # to collect column data during cleaning
checkTypes = { }
colDex = 1
for colName in dffBook.columns[ colDex: ]:
    origCol = dffBook.iloc[ :, colDex ]
    # get match if string, store if already float, catch unexpected
    if type( origCol[ 0 ] ) == str: colDict = {
        'matchedNums': matchNumbers( dffBook, colDex, patt ),
        'remainder': getRemainder( dffBook, colDex, patt ) }
    elif type( origCol[ 0 ] ) == np.float64: colDict = {
        'matchedNums': origCol.astype( float ), 'remainder': [ ] }
    elif type( origCol[ 0 ] ) != float:
        checkTypes[ type( origCol[ 0 ] ) ] = colDex
        colDict = { 'matchedNums': origCol, 'remainder': [ ] }
    else: colDict = {
        'matchedNums': matchNumbers( dffBook, colDex, patt ),
        'remainder': getRemainder( dffBook, colDex, patt ) }
    colDict[ 'origCol' ] = origCol
    dfFbDict[ colName ] = colDict
    colDex += 1

for i in checkTypes: print( f"unexpected: {i} at {checkTypes[ i ]}" )


In [6]:
# split first match item from list as float to "clean", store else to "split"
def splitFirstOther( matchList ):
    firstVals, splitVals, checkVals = [ ], [ ], [ ]
    for mNum in range( len( matchList ) ):
        el = matchList[ mNum ]
        isFilldList = (type( el ) == list) and (len( el )>0)
        if isFilldList:  # remove any thousandcomma to support convert
            firstVals.append( float( ''.join( el[ 0 ].split( ',' ) ) ) )
            splitVals.append( [ v for v in el[ 1: ] ] )
        else:  # check all else are either nan or empty matchlist
            if ((type( el ) == list and len( el )>0) and
                    (type( el ) != list and math.isnan( el ) == False)):
                checkVals.append( el )
            firstVals.append( np.nan )
            splitVals.append( np.nan )
    return firstVals, splitVals, checkVals


for colName in dfFbDict:
    colDict = dfFbDict[ colName ]
    (colDict[ 'clean' ],
    colDict[ 'splitVals' ],
    colDict[ 'checkVals' ]) = splitFirstOther( colDict[ 'matchedNums' ] )

# Raise message if got uncategorized data
for colName in dfFbDict:
    if len( dfFbDict[ colName ][ 'checkVals' ] )>0:
        print( f"Got checkvals for {colName}" )

In [7]:
# dictionary columns to DF, checking is now float
newCols = [ ]
dfFloat = dffBook.iloc[ :, 0 ]  # start with countries
for colName in dfFbDict:
    clean = pd.Series( dfFbDict[ colName ][ 'clean' ] )
    lenFloat = len( [ i for i in clean if type( i ) == float ] )
    if lenFloat>len( clean ) * 0.90:
        newCols.append( colName )
        dfFloat = pd.concat( [ dfFloat, clean ], axis=1 )
    else: print( "col is less than 90% float. Dropping..." )

dfFloat.columns = [ 'Country' ] + newCols
dfFloat


,Country,url,Introduction: Background,Geography: Location,Geography: Geographic coordinates,Geography: Map references,Geography: Area - total,Geography: Area - land,Geography: Area - water,Geography: Area - comparative,...,Transportation: Waterways - note 2,Transportation: Waterways - top ten largest natural lakes (by surface area),Transportation: Waterways - note 3,Transportation: Ports and terminals - top twenty container ports as measured by Twenty-Foot Equivalent Units (TEUs) throughput,Transnational Issues: Refugees and internally displaced persons,Transnational Issues: Trafficking in persons - Tier 2 Watch List,Transnational Issues: Trafficking in persons - Tier 3,Transnational Issues: Illicit drugs - cocaine,Transnational Issues: Illicit drugs - opiates,Energy: Electricity access - population without electricity
0,Afghanistan,NaN,174.0,NaN,33.0,NaN,652230.000,652230.00,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Akrotiri,NaN,196.0,NaN,34.0,NaN,123.000,NaN,NaN,0.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Albania,NaN,191.0,NaN,41.0,NaN,28748.000,27398.00,1350.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Algeria,NaN,3.0,NaN,28.0,NaN,2381740.000,2381740.00,0.0,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,American Samoa,NaN,100.0,NaN,14.0,NaN,224.000,224.00,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,West Bank,NaN,15.0,NaN,32.0,NaN,5860.000,5640.00,220.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254,World,NaN,20.0,NaN,NaN,NaN,510.072,148.94,361.9,16.0,...,20.0,372960.0,1620.0,43303000.0,202.0,44.0,19.0,202.0,265000.0,NaN
255,Yemen,NaN,191.0,NaN,15.0,NaN,527968.000,527968.00,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0
256,Zambia,NaN,188.0,NaN,15.0,NaN,752618.000,743398.00,9220.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Enforce non-nan threshold for rows and columns

def nanThreshold( notNan ):  # average plus .5 standard deviation (rounded)
    return int( (sum( notNan ) / len( notNan )) + 0.5 * np.std( notNan ) )


def nonNanFromDims( dfr, dim = 1 ):
    nonNans = [ ]
    for pos in range( 0, dfr.shape[ dim ] ):
        vals = (dfr.iloc[ :, pos ].tolist() if dim == 1
                else dfr.loc[ [ pos ] ].values.tolist()[ 0 ])
        
        nonNans.append( [ vals, len( [ v for v in vals if (
                type( v ) == float and not math.isnan( v )) ] ) ] )
    
    _thresh = nanThreshold( [ nval for _, nval in nonNans ] )
    keepVals = [ kval for kval, nnul in nonNans if nnul>=_thresh ]
    print( f"non-nan[ {len( keepVals )} ] thr[ {_thresh} ] dim[ {dim} ]" )
    
    return keepVals

In [9]:
# ROWS
dfRowsClean = pd.DataFrame( nonNanFromDims( dfFloat, dim=0 ) )

# add a columnindex row to track names of kept columns
dfRowsClean.loc[ -1 ] = dfFloat.columns
dfRowsClean.index = dfRowsClean.index + 1
dfRowsClean.sort_index( inplace=True )
dfRowsClean


non-nan[ 125 ] thr[ 255 ] dim[ 0 ]


,0,1,2,3,4,5,6,7,8,9,...,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053
0,Country,url,Introduction: Background,Geography: Location,Geography: Geographic coordinates,Geography: Map references,Geography: Area - total,Geography: Area - land,Geography: Area - water,Geography: Area - comparative,...,Transportation: Waterways - note 2,Transportation: Waterways - top ten largest na...,Transportation: Waterways - note 3,Transportation: Ports and terminals - top twen...,Transnational Issues: Refugees and internally ...,Transnational Issues: Trafficking in persons -...,Transnational Issues: Trafficking in persons -...,Transnational Issues: Illicit drugs - cocaine,Transnational Issues: Illicit drugs - opiates,Energy: Electricity access - population withou...
1,Afghanistan,NaN,174.0,NaN,33.0,NaN,652230.0,652230.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Albania,NaN,191.0,NaN,41.0,NaN,28748.0,27398.0,1350.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Algeria,NaN,3.0,NaN,28.0,NaN,2381740.0,2381740.0,0.0,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Angola,NaN,14.0,NaN,12.0,NaN,1246700.0,1246700.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,Venezuela,NaN,183.0,NaN,8.0,NaN,912050.0,882050.0,30000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
122,Vietnam,NaN,10.0,NaN,16.0,NaN,331210.0,310070.0,21140.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,Yemen,NaN,191.0,NaN,15.0,NaN,527968.0,527968.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0
124,Zambia,NaN,188.0,NaN,15.0,NaN,752618.0,743398.0,9220.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# COLS
keepCols = nonNanFromDims( dfRowsClean, dim=1 )

# convert to numeric df
dfColsClean = pd.DataFrame( { col[ 0 ]: col[ 1: ] for col in keepCols } )
df = dfColsClean.apply( pd.to_numeric, errors='ignore' )

non-nan[ 267 ] thr[ 57 ] dim[ 1 ]


In [11]:
# final type check & add countries
dTypeDict = dfColsClean.dtypes
for itemName in list( dTypeDict.keys() ):
    if dTypeDict[ itemName ] != np.float64: print( f"NOT FLOAT: {itemName}" )
dfColsClean.insert( 0, 'Country', dfRowsClean.iloc[ :, 0 ].tolist()[ 1: ] )
dfColsClean

,Country,Introduction: Background,Geography: Geographic coordinates,Geography: Area - total,Geography: Area - land,Geography: Area - water,Geography: Land boundaries - total,Geography: Coastline,Geography: Elevation - highest point,Geography: Elevation - lowest point,...,Transportation: Railways - standard gauge,Transportation: Merchant marine - total,Transportation: Merchant marine - by type,Transnational Issues: Refugees and internally displaced persons - stateless persons,Environment: Waste and recycling - municipal solid waste recycled annually,Environment: Waste and recycling - percent of municipal solid waste recycled,Transportation: Railways - narrow gauge,Geography: Maritime claims - exclusive economic zone,Economy: Credit ratings - Fitch rating,Military and Security: Military deployments
0,Afghanistan,174.0,33.0,652230.0,652230.0,0.0,5987.00,0.0,7492.0,258.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Albania,191.0,41.0,28748.0,27398.0,1350.0,691.00,362.0,2764.0,0.0,...,677.0,70.0,47.0,1528.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Algeria,3.0,28.0,2381740.0,2381740.0,0.0,6734.00,998.0,2908.0,-40.0,...,2888.0,114.0,1.0,NaN,990299.0,8.0,1085.0,NaN,NaN,NaN
3,Angola,14.0,12.0,1246700.0,1246700.0,0.0,5369.00,1600.0,2620.0,0.0,...,NaN,54.0,13.0,NaN,NaN,NaN,2729.0,200.0,202.0,NaN
4,Argentina,181.0,34.0,2780400.0,2736690.0,43710.0,11968.00,4989.0,6962.0,-105.0,...,2745.1,202.0,1.0,NaN,1074633.0,6.0,7523.3,200.0,202.0,200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,Venezuela,183.0,8.0,912050.0,882050.0,30000.0,5267.00,2800.0,4978.0,0.0,...,447.0,281.0,4.0,NaN,NaN,NaN,NaN,200.0,201.0,NaN
121,Vietnam,10.0,16.0,331210.0,310070.0,21140.0,4616.00,3444.0,3144.0,0.0,...,178.0,1926.0,116.0,30581.0,2201169.0,23.0,2169.0,200.0,201.0,NaN
122,Yemen,191.0,15.0,527968.0,527968.0,0.0,1601.00,1906.0,3666.0,0.0,...,NaN,34.0,2.0,NaN,386946.0,8.0,NaN,200.0,NaN,NaN
123,Zambia,188.0,15.0,752618.0,743398.0,9220.0,6043.15,0.0,2330.0,329.0,...,NaN,2.0,1.0,NaN,NaN,NaN,3126.0,NaN,202.0,925.0


In [12]:
# Review clean DF
fbIsNa = dffBook.isna().sum().sum()
dfIsNa = dfColsClean.isna().sum().sum()
fbDim = dffBook.shape[ 0 ] * dffBook.shape[ 1 ]
dfDim = dfColsClean.shape[ 0 ] * dfColsClean.shape[ 1 ]
print( f"factbook originally shape: {dffBook.shape}" )
print( f"    NAN-density: {(fbIsNa / fbDim):.2f}% "
       f"({fbIsNa} NaN in {fbDim})" )
print( f"clean dataframe has shape: {dfColsClean.shape}" )
print( f"    NAN-density: {(dfIsNa / dfDim):.2f}% "
       f"({dfIsNa} NaN in {dfDim})" )



factbook originally shape: (258, 1054)
    NAN-density: 0.74% (200708 NaN in 271932)
clean dataframe has shape: (125, 268)
    NAN-density: 0.05% (1796 NaN in 33500)


In [13]:
# Reporting to identify any differences of unit scale within features

cleanReman = { }

cleanPatts = [
    r'(\([^)]*\))',  # remove all bracketed
    r'(.* m)$|(.* m) '  # remove all before " m[endline/space]" (mtn names)
    ]

for colName in list( dfColsClean.columns )[ 1: ]:
    remainder = dfFbDict[ colName ][ 'remainder' ]
    for pattStr in cleanPatts:
        patt = re.compile( pattStr )
        remainder = remainder.str.replace( patt, '' )
    cleanReman.update( { colName: remainder } )

In [14]:
# CHECK FOR SCALE FACTORS

def runScaleAnalysis( dfr ):
    colList = list( dfr.columns )
    dropFeatrs = [ ]
    cleanNotes = { }
    
    for pos in range( 1, len( colList ) ):
        colNam = colList[ pos ]
        colVals = dfr.iloc[ :, pos ].tolist()[ :10 ]
        remndr = set( cleanReman[ colNam ] )
        rMainPrint = ""
        for r in list( remndr )[ :25 ]:
            if type( r ) == float: rMainPrint = rMainPrint + f"{r}\n"
            else: rMainPrint = rMainPrint + f"{r[ :60 ]}\n"
        
        report = (
            f"COL [ {pos} ] {colNam}\n\n"
            f"CLEANVALS:\n{colVals}\n\n"
            f"REMAINDER (unq in col: {len( remndr )}):\n{rMainPrint}\n")
        
        report_a = report + "\nACCEPT(A), BREAK(B), CLEAN/SCALE NOTE(C), DROP(D)"
        report_b = report_a + "\n\nPLEASE MAKE A SELECTION:\n\n"
        usinp = input( report_a )
        while usinp not in [ 'a', 'd', 'c', 'b' ]: usinp = input( report_b )
        if usinp == 'b': break
        elif usinp == 'a': continue
        elif usinp == 'd': dropFeatrs.append( colNam )
        else: cleanNotes.update( { colNam:
            input( f"{report[ :250 ]}...\n\n\nCLEANING/SCALE NOTE" ) } )
    
    return dropFeatrs, cleanNotes

# dropFeatrs, cleanNotes = runScaleAnalysis(dfColsClean)

In [15]:
# # SAVE SCALE ANALYSIS DATA
# stmp = fTools.dtStamp()
# fTools.storePKL( dropFeatrs, f'dropFeatrs_{stmp}', currWorkDir, subdir=None )
# fTools.storePKL( cleanNotes, f'cleanNotes_{stmp}', currWorkDir, subdir=None )

In [16]:
# LOAD SCALE ANALYSIS DATA
pklFiles = [ fi for fi in [ open( pth, 'rb' )
    for pth in [ list( dKey )[ 0 ]
        for dKey in [ fTools.datesortFiles( currWorkDir, fNam )
            for fNam in [ 'dropFeatrs', 'cleanNotes' ]
            ] ] ] ]

dropFeats, scaleNotes = [ pickle.load( fi ) for fi in pklFiles ]
for fi in pklFiles: fi.close()


In [17]:
# update value by matching remainder scale fragment via country reference
df = dfColsClean.copy()
scaleDict = {
    "million": 1000000,
    "billion": 1000000000,
    "trillion": 1000000000000
    }

# updatedVals = {}

tested = [ ]

cleanCountries = list( df[ 'Country' ] )

for colName in list( scaleNotes.keys() )[ :12 ]:
    remnCol = dfFbDict[ colName ][ 'remainder' ]
    row = 0
    
    # checking remnantcol (HAS PRE-CLEAN ENTRIES) for match
    for remnt in remnCol:
        if type( remnt ) == float: row += 1; continue
        country = dffBook[ 'Country' ][ row ]
        if country not in cleanCountries: row += 1; continue
        
        matches = [ ]
        
        # apply scale of lowest-index matched scale-string in remnant
        for scale in scaleDict:
            try: matches.append( [ remnt.index( scale ), scale ] )
            except ValueError: continue
        
        if len( matches )>0:  # sort by lowest index (first val of match)
            matchScale = sorted( matches, key=lambda x: x[ 0 ] )[ 0 ][ 1 ]
            val = df.loc[ df[ 'Country' ] == country ][ colName ].iloc[ 0 ]
            newVal = val * scaleDict[matchScale]
            
            if matchScale not in tested:
                tested.append( matchScale )
                print( f"in {colName[ :15 ]}:  in {country}, " )
                print( F"COL: {colName}\n"
                       F"ROW: {row} ({country})\n"
                       F"VAL: {val}\n"
                       F"RMT: '{remnt}'\n"
                       F"SCL: {matchScale}\n"
                       F"NVL: {newVal}\n" )
                
            # new df from vals scaled where match
        
        row += 1

in Geography: Area:  in Chad, 
COL: Geography: Area - total
ROW: 46 (Chad)
VAL: 1.284
RMT: ' million sq km'
SCL: million
NVL: 1284000.0

in Environment: To:  in Algeria, 
COL: Environment: Total water withdrawal - municipal
ROW: 3 (Algeria)
VAL: 3.6
RMT: ' billion cubic meters ( est.)'
SCL: billion
NVL: 3600000000.0

in Environment: To:  in Brazil, 
COL: Environment: Total renewable water resources
ROW: 32 (Brazil)
VAL: 8.647
RMT: ' trillion cubic meters ( est.)'
SCL: trillion
NVL: 8647000000000.0



In [18]:
#END